In [1]:
import requests
import re
import pdfplumber
import pandas as pd
import os
import io
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import warnings

warnings.filterwarnings("ignore", message="CropBox missing from /Page, defaulting to MediaBox")


In [2]:
# get df with all links
year = 2024
mainurl = f'https://www.cmdachennai.gov.in/ApprovedLayout/{year}.htm'
df = pd.read_html(mainurl)[0]
df = df.iloc[:-1,:-1]


resp = requests.get(mainurl)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, 'html.parser')
pdflist = soup.find_all('a')
pdflist = [i for i in pdflist if i.get('href').endswith('.pdf')]
df['bs4'] = pdflist

def extractpplfromurl(url):
    return url.split('/')[-1][:-4].replace('-','/')

df['url'] = df.bs4.apply(lambda x: 'https://www.cmdachennai.gov.in/' + x.get('href'))
df['extracted_ppl'] = df.url.apply(extractpplfromurl)
df['pdf_savepath'] = df.extracted_ppl.apply(lambda x: x.split('/')[-1] +'/'+ x.split('/')[0] + '.pdf' )


In [7]:
# verify layout numbers match
pd.DataFrame(df.iloc[:,1] == df.iloc[:,-2]).all()



0    True
dtype: bool

In [ ]:
async def downlaod_and_extract_pdf(session, df_row, x_start_ratio=0.4):
    pdf_url, savepath = df_row['url'], 'data/' + df_row['pdf_savepath']
    try:
        all_text = []
        savedtodisk = False
        # Use aiohttp for async HTTP requests
        async with session.get(pdf_url, ssl=False) as response:
            response.raise_for_status()
            content = await response.read()
            
            # Read PDF
            pdf_file = io.BytesIO(content)
            # save the pdf file
            os.makedirs(os.path.dirname(os.path.abspath(savepath)), exist_ok=True)
            with open(savepath, 'wb') as fh:
                fh.write(content)
                savedtodisk = True
                print(f"Saved PDF to disk: {savepath}")

            with pdfplumber.open(pdf_file) as pdf:
                for page in pdf.pages:
                    width = page.width
                    height = page.height
                    
                    # Dynamically compute the crop box based on width
                    # Adjust the start ratio (0.4 or 0.5) as needed
                    crop = page.within_bbox((width * x_start_ratio, 0, width, height))
                    text = crop.extract_text()
                    if text:
                        all_text.append(text)
        return "\n".join(all_text), savedtodisk
    except Exception as e:
        return f"ERROR: {e}", False
    

def extract_multiple_patterns(text: str) -> dict[str|None]:
    """
    Extract multiple patterns from PDF text
    
    Returns:
        Dictionary with pattern matches
    """
    # Define patterns with descriptive keys
    patterns = {
        'total_no_plots': [
            r'NO\.?\s*OF\.?\s*PLOTS\s*[=:]\s*(\d+)',
            r'No\.?\s*of\s*[Pp]lots\s*[=:]\s*(\d+)',
            r'TOTAL\s*PLOTS?\s*[=:]\s*(\d+)'
        ],
        'ews': [
            r'(?i)\bE\.?W\.?S\.?\s+(?:Provided|Plots)\b[^\n]*[:=]\s*(\d+)\s*Nos\.?\b'
        ],
        'regular':[
            r'(?i)REGULAR PLOTS.\s*[^\n]*?[:=]\s*(\d+)\s*No'
        ],
        'date': [
            r'DATE\s*[=:]\s*(\d{1,2}\s?[-\/]\s?\d{2}\s?[-\/]\s?\d{4})',
            r'Date\s*:\s*(\d{1,2}(?:st|nd|rd|th)?\s+(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+\d{4})',
            r'(\d{4}-\d{2}-\d{2})'
        ],
        'reference_number': [
            r'REF(?:ERENCE)?\s*NO\.?\s*[=:]\s*([A-Z0-9-/]+)',
            r'File\s*No\.?\s*[=:]\s*([A-Z0-9-/]+)',
            r'Application\s*No\.?\s*[=:]\s*([A-Z0-9-/]+)'
        ]
    }

    try:
        
        # Dictionary to store results
        results = {}
        
        # Search for each pattern category
        for category, pattern_list in patterns.items():
            results[category] = None
            for pattern in pattern_list:
                matches = re.findall(pattern, text, re.IGNORECASE)
                if matches:
                    results[category] = matches[0]
                    break
        
        return results
        
    except Exception as e:
        print(f"Error processing URL {text}: {str(e)}")
        return {key: None for key in patterns.keys()}, text


In [32]:
# Now populate the columns from the extracted patterns
def extract_and_expand(text):
    if text and not isinstance(text, str) or (isinstance(text, str) and not text.startswith("ERROR:")):
        patterns = extract_multiple_patterns(text)
        return patterns
    return {key: None for key in ['total_no_plots', 'ews', 'regular', 'date', 'reference_number']}

# Usage with asyncio
async def process_df(df):
    results_text = []
    results_saved = []
    async with aiohttp.ClientSession() as session:
        tasks = [downlaod_and_extract_pdf(session, row) for _, row in df.iterrows()]
        batch_results = await asyncio.gather(*tasks)
        
        # Unpack results
        batch_text, batch_saved = zip(*batch_results)
        results_text.extend(batch_text)
        results_saved.extend(batch_saved)
        
    result_df = df.copy()
    result_df['extracted_text'] = results_text
    result_df['saved_to_disk'] = results_saved
    return result_df


async def main():
    # Process first 20 rows as an example
    result_df = await process_df(df)
    
    # Now you can use the DataFrame as usual
    print(f"Successfully saved {result_df['saved_to_disk'].sum()} of {len(result_df)} PDFs")
    
    # Apply pattern extraction to the extracted text and expand into separate columns
    for key in ['total_no_plots', 'ews', 'regular', 'date', 'reference_number']:
        result_df[key] = None  # Initialize columns


    # Apply the extraction function
    pattern_dicts = result_df['extracted_text'].apply(extract_and_expand)

    # Expand the dictionaries into separate columns
    for key in ['total_no_plots', 'ews', 'regular', 'date', 'reference_number']:
        result_df[key] = pattern_dicts.apply(lambda x: x.get(key))
    
    return result_df

# Run the async function
result_df = await main()

CropBox missing from /Page, defaulting to MediaBox


Saved PDF to disk: data/2024/97.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Saved PDF to disk: data/2024/75.pdf
Saved PDF to disk: data/2024/47.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


...

Saved PDF to disk: data/2024/181.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Saved PDF to disk: data/2024/276.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Saved PDF to disk: data/2024/178.pdf
Successfully saved 275 of 309 PDFs


In [35]:
pqdf = result_df.drop(columns='bs4')
pqdf.to_parquet('./2024extract.parquet')

In [2]:
pqdf = pd.read_parquet('./2024extract.parquet')
pqdf[pqdf.extracted_text.str.startswith("ERROR")]

,SNo.,Approval No. PPD/L.O. No.,Localbody,Village,url,extracted_ppl,pdf_savepath,extracted_text,saved_to_disk,total_no_plots,ews,regular,date,reference_number
6,7,07/2024,Thiruverkadu MP,Noombal,https://www.cmdachennai.gov.in/pdfs/layouts/20...,07/2024,2024/07.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
16,17,17/2024,Sholavaram PU,Nayar II,https://www.cmdachennai.gov.in/pdfs/layouts/20...,17/2024,2024/17.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
36,37,37/2024,St.Thomas Mount PU,Kaspapuram,https://www.cmdachennai.gov.in/pdfs/layouts/20...,37/2024,2024/37.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
69,70,70/2024,Kundrathur PU,Kulathuvancheri,https://www.cmdachennai.gov.in/pdfs/layouts/20...,70/2024,2024/70.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
80,81,81/2024,Kundrathur PU,Kovur,https://www.cmdachennai.gov.in/pdfs/layouts/20...,81/2024,2024/81.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
93,94,94/2024,Poonamallee PU,"Sithukkadu,",https://www.cmdachennai.gov.in/pdfs/layouts/20...,94/2024,2024/94.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
94,95,95/2024,St.Thomas Mount PU,Agaramthen,https://www.cmdachennai.gov.in/pdfs/layouts/20...,95/2024,2024/95.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
127,128,128/2024,St.Thomas Mount PU,Vengambakkam,https://www.cmdachennai.gov.in/pdfs/layouts/20...,128/2024,2024/128.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
129,130,130/2024,Puzhal PU,Vilangadupakkam,https://www.cmdachennai.gov.in/pdfs/layouts/20...,130/2024,2024/130.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
130,131,131/2024,St.Thomas Mount PU,Arasankazhani,https://www.cmdachennai.gov.in/pdfs/layouts/20...,131/2024,2024/131.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None


In [4]:
pqdf[pqdf.total_no_plots.isna() & pqdf.saved_to_disk]

,SNo.,Approval No. PPD/L.O. No.,Localbody,Village,url,extracted_ppl,pdf_savepath,extracted_text,saved_to_disk,total_no_plots,ews,regular,date,reference_number
35,36,36/2024,Poonamallee PU,Vellavedu,https://www.cmdachennai.gov.in/pdfs/layouts/20...,36/2024,2024/36.pdf,FILE NO: LAYOUT-1 / 0221 / 2023\nNT = 2347SQ.M...,True,None,None,None,21 / 02 / 2024,LAYOUT-1
37,38,38/2024,Poonamallee PU,Kannapalayam,https://www.cmdachennai.gov.in/pdfs/layouts/20...,38/2024,2024/38.pdf,: 3075 SQ.M\n(AS PER PATTA)\n: 1279 SQ.M\nOSE ...,True,None,None,None,23 / 02 / 2024,None
42,43,43/2024,Sholavaram PU,Angadu,https://www.cmdachennai.gov.in/pdfs/layouts/20...,43/2024,2024/43.pdf,= 2300SQ.M FILE NO: LAYOUT-1 / 0230 / 2022\n= ...,True,None,None,None,28 / 02 / 2024,LAYOUT-1
49,50,50/2024,Kundrathur PU,Tharapakkam,https://www.cmdachennai.gov.in/pdfs/layouts/20...,50/2024,2024/50.pdf,FILE NO: LAYOUT-1 / 0158 / 2023\n(cid:54)IT(ci...,True,None,None,None,11 / 03 / 2024,LAYOUT-1
55,56,56/2024,St.Thomas Mount PU,Thiruvanchery,https://www.cmdachennai.gov.in/pdfs/layouts/20...,56/2024,2024/56.pdf,FILE NO: LAYOUT-1 / 0187 / 2023\nTENT = 2590SQ...,True,None,None,None,12 / 03 / 2024,LAYOUT-1
56,57,57/2024,Poonamallee PU,Ayalchari,https://www.cmdachennai.gov.in/pdfs/layouts/20...,57/2024,2024/57.pdf,FILE NO: LAYOUT-1 / 0269 / 2022\n(cid:54)ITE E...,True,None,None,None,12 / 03 / 2024,LAYOUT-1
57,58,58/2024,Thirumazhisai TP,Thirumazhisai,https://www.cmdachennai.gov.in/pdfs/layouts/20...,58/2024,2024/58.pdf,= 4694SQ.M FILE NO: LAYOUT-1 / 0134 / 2023\n= ...,True,None,None,None,14 / 03 / 2024,LAYOUT-1
61,62,62/2024,Tambaram Corporation,Tambaram,https://www.cmdachennai.gov.in/pdfs/layouts/20...,62/2024,2024/62.pdf,FILE NO.L1/ 249/2023\nTAL EXTENT (AS PER PATTA...,True,None,None,None,20 / 03 / 2024,None
66,67,67/2024,Poonamallee PU,Korattur,https://www.cmdachennai.gov.in/pdfs/layouts/20...,67/2024,2024/67.pdf,FILE NO.L1/0306/2023\nAL EXTENT (AS PER DOCUEN...,True,None,None,None,05 / 04 / 2024,None
70,71,71/2024,Villivakkam PU,Palavedu,https://www.cmdachennai.gov.in/pdfs/layouts/20...,71/2024,2024/71.pdf,T : 1200 SQ.M\n(AS PER PATTA)\n: 32 SQ.M\nS : ...,True,None,None,None,08 / 04 / 2024,None


In [8]:
import glob
for i in (glob.glob('./data/2024/*.pdf')):
    print(i)

./data/2024/61.pdf
./data/2024/113.pdf
./data/2024/77.pdf
./data/2024/220.pdf
./data/2024/213.pdf
./data/2024/169.pdf
./data/2024/10.pdf
./data/2024/74.pdf
./data/2024/279.pdf
./data/2024/257.pdf
./data/2024/28.pdf
./data/2024/62.pdf
./data/2024/269.pdf
./data/2024/64.pdf
./data/2024/110.pdf
./data/2024/30.pdf
./data/2024/87.pdf
./data/2024/240.pdf
./data/2024/166.pdf
./data/2024/164.pdf
./data/2024/91.pdf
./data/2024/172.pdf
./data/2024/141.pdf
./data/2024/147.pdf
./data/2024/219.pdf
./data/2024/249.pdf
./data/2024/291.pdf
./data/2024/263.pdf
./data/2024/242.pdf
./data/2024/23.pdf
./data/2024/52.pdf
./data/2024/163.pdf
./data/2024/216.pdf
./data/2024/168.pdf
./data/2024/140.pdf
./data/2024/255.pdf
./data/2024/160.pdf
./data/2024/262.pdf
./data/2024/126.pdf
./data/2024/188.pdf
./data/2024/105.pdf
./data/2024/209.pdf
./data/2024/42.pdf
./data/2024/115.pdf
./data/2024/302.pdf
./data/2024/129.pdf
./data/2024/238.pdf
./data/2024/308.pdf
./data/2024/273.pdf
./data/2024/144.pdf
./data/2024/2

In [3]:
# adaptive extract 
import glob
# take everything and see how it looks like
found, notfound = [], []
for path in sorted(glob.glob('./data/2024/*.pdf')):
    all_text = []
    extentfound=[]
    try:
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                width = page.width
                height = page.height
                
                # Dynamically compute the crop box based on width
                # Adjust the start ratio (0.4 or 0.5) as needed
                words = page.extract_words()
                extentfound = [i for i in words if i['text'].lower() == 'extent']

                # crop = page.within_bbox((width * 0, 0, width, height))
                # text = crop.extract_text()
                # if text:
                #     all_text.append(text)
    except:
        pass
    finally:
        if extentfound:
            print(f'{path}, found extent in pdf')
            # print(extentfound[0])
            print('-----------------')
            found.append(path)
        else:
            print(f'{path}, NO')
            # print(extentfound[0])
            print('-----------------')
            notfound.append(path)

# print('/n'.join(all_text))


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


./data/2024/01.pdf, found extent in pdf
-----------------


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


./data/2024/02.pdf, found extent in pdf
-----------------
./data/2024/03.pdf, found extent in pdf
-----------------


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


...

./data/2024/96.pdf, found extent in pdf
-----------------


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


./data/2024/97.pdf, found extent in pdf
-----------------


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


./data/2024/98.pdf, found extent in pdf
-----------------
./data/2024/99.pdf, found extent in pdf
-----------------


In [4]:
notfound

['./data/2024/14.pdf',
 './data/2024/181.pdf',
 './data/2024/198.pdf',
 './data/2024/50.pdf',
 './data/2024/57.pdf']